In [1]:
pip install feedparser requests beautifulsoup4 typing_extensions openai


Note: you may need to restart the kernel to use updated packages.


In [43]:
import feedparser
import requests
from bs4 import BeautifulSoup
import openai
import os

def extract_article_text(url):
    """Fetch and extract full article text from the given blog post URL."""
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
    except requests.RequestException as e:
        return f"Error fetching article: {e}"

    soup = BeautifulSoup(response.content, 'html.parser')

    # Target the article content using the actual class from NVIDIA's blog
    article_div = soup.find('div', class_='entry-content')
    if article_div:
        return article_div.get_text(separator='\n', strip=True)
    else:
        return "Article content not found."
        
# Set your API key
#os.environ["OPENAI_API_KEY"] = "..."

from openai import OpenAI

# Instantiate the client
client = OpenAI()

def get_companies_mentioned(text):
    prompt = f"For the text below give me only a list of publicly traded companies mentioned in it :\n\n{text}"

    completion = client.chat.completions.create(
        model="gpt-4.1-nano",  # or "gpt-3.5-turbo"
        messages=[
            {"role": "user", "content": prompt}
        ],
        max_tokens=150,
        temperature=0,
    )

    return completion.choices[0].message.content.strip()

def fetch_latest_articles(feed_url, count=5):
    """Parse RSS feed and extract top N article URLs and titles."""
    feed = feedparser.parse(feed_url)
    articles = feed.entries[:count]
    
    for idx, article in enumerate(articles, 1):
        title = article.title
        link = article.link
        print(f"\n📄 Article {idx}: {title}")
        print(f"🔗 URL: {link}\n")

        # Extract and print the article text
        text = extract_article_text(link)
        #print(f"📝 Content:\n{text}\n{'='*80}")

        # Get companies mentioned using OpenAI
        companies = get_companies_mentioned(text)
        print(f"🏢 Publicly traded companies mentioned:\n{companies}")
        print("="*80)

# NVIDIA blog RSS feed
rss_feed_url = 'https://blogs.nvidia.com/feed/'

# Run the process for the latest 5 articles
fetch_latest_articles(rss_feed_url, count=5)



📄 Article 1: Game On With GeForce NOW, the Membership That Keeps on Delivering
🔗 URL: https://blogs.nvidia.com/blog/geforce-now-thursday-elder-scrolls-online-member-reward/

🏢 Publicly traded companies mentioned:
NVIDIA

📄 Article 2: Startup Uses NVIDIA RTX-Powered Generative AI to Make Coolers, Cooler
🔗 URL: https://blogs.nvidia.com/blog/rtx-ai-garage-fity-flex-flux-comfyui-stable-diffusion/

🏢 Publicly traded companies mentioned:
NVIDIA

📄 Article 3: Into the Omniverse: World Foundation Models Advance Autonomous Vehicle Simulation and Safety
🔗 URL: https://blogs.nvidia.com/blog/wfm-advance-av-sim-safety/

🏢 Publicly traded companies mentioned:
- NVIDIA
- Uber

📄 Article 4: HPE and NVIDIA Debut AI Factory Stack to Power Next Industrial Shift
🔗 URL: https://blogs.nvidia.com/blog/hpe-nvidia-ai-factory/

🏢 Publicly traded companies mentioned:
- HPE (Hewlett Packard Enterprise)
- NVIDIA (NVIDIA Corporation)
- KDDI

📄 Article 5: NVIDIA and Partners Highlight Next-Generation Robotics, Auto